In [ ]:
# ==== System info ====
!nvidia-smi -L || echo "CPU-only runtime"
!python --version

# ==== Install AutoGluon and Kaggle ====
!pip install -U pip
!pip install -U "autogluon>=1.0" kaggle

# ==== Ensure folders exist ====
import os
for d in ["data","artifacts"]:
    os.makedirs(d, exist_ok=True)
print("Setup complete. Remember: Edit ▸ Notebook settings ▸ uncheck 'Omit code cell output when saving'.")


In [ ]:
# ---- Load dataset (scikit-learn California Housing) ----
from sklearn.datasets import fetch_california_housing
import pandas as pd
from autogluon.tabular import TabularPredictor

cal = fetch_california_housing(as_frame=True)
df = cal.frame.copy()
df.rename(columns={'MedHouseVal': 'SalePrice'}, inplace=True)

train = df.sample(frac=0.8, random_state=42)
test  = df.drop(train.index)
print("Train/Test shapes:", train.shape, test.shape)


In [ ]:
# ---- Train predictor ----
predictor = TabularPredictor(label='SalePrice', problem_type='regression', path='ag_cal/').fit(
    train_data=train,
    time_limit=900,                  # increase if you can
    presets='medium_quality_faster_train'   # try 'best_quality' if you have time
)
print("Training complete.")


In [ ]:
# ---- Evaluate & export artifacts ----
lb = predictor.leaderboard(test, silent=True)
fi = predictor.feature_importance(test)

lb_path = 'artifacts/cal_housing_leaderboard.csv'
fi_path = 'artifacts/cal_housing_feature_importance.csv'
lb.to_csv(lb_path, index=False)
fi.to_csv(fi_path)

preds = predictor.predict(test.drop(columns=['SalePrice']))
out = pd.concat([test[['SalePrice']].reset_index(drop=True), preds.rename('Predicted')], axis=1)

print("Saved:", lb_path, fi_path)
lb.head(10), fi.head(10), out.head(10)
